# Lab | Data Aggregation and Filtering

In this challenge, we will continue to work with customer data from an insurance company. We will use the dataset called marketing_customer_analysis.csv, which can be found at the following link:

https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis.csv

This dataset contains information such as customer demographics, policy details, vehicle information, and the customer's response to the last marketing campaign. Our goal is to explore and analyze this data by first performing data cleaning, formatting, and structuring.

In [1]:
import pandas as pd
data_mk_cust_analysis=pd.read_csv("https://raw.githubusercontent.com/data-bootcamp-v4/data/main/marketing_customer_analysis_clean.csv")

1. Create a new DataFrame that only includes customers who have a total_claim_amount greater than $1,000 and have a response of "Yes" to the last marketing campaign.

In [2]:
data_mk_cust_analysis.head()

,unnamed:_0,customer,state,customer_lifetime_value,response,coverage,education,effective_to_date,employmentstatus,gender,...,number_of_policies,policy_type,policy,renew_offer_type,sales_channel,total_claim_amount,vehicle_class,vehicle_size,vehicle_type,month
0,0,DK49336,Arizona,4809.216960,No,Basic,College,2011-02-18,Employed,M,...,9,Corporate Auto,Corporate L3,Offer3,Agent,292.800000,Four-Door Car,Medsize,A,2
1,1,KX64629,California,2228.525238,No,Basic,College,2011-01-18,Unemployed,F,...,1,Personal Auto,Personal L3,Offer4,Call Center,744.924331,Four-Door Car,Medsize,A,1
2,2,LZ68649,Washington,14947.917300,No,Basic,Bachelor,2011-02-10,Employed,M,...,2,Personal Auto,Personal L3,Offer3,Call Center,480.000000,SUV,Medsize,A,2
3,3,XL78013,Oregon,22332.439460,Yes,Extended,College,2011-01-11,Employed,M,...,2,Corporate Auto,Corporate L3,Offer2,Branch,484.013411,Four-Door Car,Medsize,A,1
4,4,QA50777,Oregon,9025.067525,No,Premium,Bachelor,2011-01-17,Medical Leave,F,...,7,Personal Auto,Personal L2,Offer1,Branch,707.925645,Four-Door Car,Medsize,A,1


In [6]:
data_mk_cust_analysis_col=[list(data_mk_cust_analysis.columns)]
display(data_mk_cust_analysis_col)
display(data_mk_cust_analysis.dtypes)

[['unnamed:_0',
  'customer',
  'state',
  'customer_lifetime_value',
  'response',
  'coverage',
  'education',
  'effective_to_date',
  'employmentstatus',
  'gender',
  'income',
  'location_code',
  'marital_status',
  'monthly_premium_auto',
  'months_since_last_claim',
  'months_since_policy_inception',
  'number_of_open_complaints',
  'number_of_policies',
  'policy_type',
  'policy',
  'renew_offer_type',
  'sales_channel',
  'total_claim_amount',
  'vehicle_class',
  'vehicle_size',
  'vehicle_type',
  'month']]

unnamed:_0                         int64
customer                          object
state                             object
customer_lifetime_value          float64
response                          object
coverage                          object
education                         object
effective_to_date                 object
employmentstatus                  object
gender                            object
income                             int64
location_code                     object
marital_status                    object
monthly_premium_auto               int64
months_since_last_claim          float64
months_since_policy_inception      int64
number_of_open_complaints        float64
number_of_policies                 int64
policy_type                       object
policy                            object
renew_offer_type                  object
sales_channel                     object
total_claim_amount               float64
vehicle_class                     object
vehicle_size    

In [3]:
#checking
print(data_mk_cust_analysis.shape)
print(data_mk_cust_analysis['total_claim_amount'].min())

(10910, 27)
0.099007


In [7]:
# Keeping only lines where total_claim_amount is greater than $1,000
filtered_data_step1 = data_mk_cust_analysis.loc[data_mk_cust_analysis['total_claim_amount'] > 1000]

In [9]:
#checking
print(filtered_data_step1.shape)
print(filtered_data_step1['total_claim_amount'].min())

(491, 27)
1000.8


In [11]:
print(data_mk_cust_analysis['response'].unique())

['No' 'Yes']


In [13]:
# Keeping only lines where customers replied Yes
filtered_data_step2 =filtered_data_step1.loc[filtered_data_step1['response']== "Yes"]

In [14]:
#checking
print(filtered_data_step2.shape)
print(filtered_data_step2['response'].unique())

(67, 27)
['Yes']


2. Using the original Dataframe, analyze the average total_claim_amount by each policy type and gender for customers who have responded "Yes" to the last marketing campaign. Write your conclusions.

In [16]:
# Filter first to keep only data where customers answered "Yes"
filtered_data = data_mk_cust_analysis[data_mk_cust_analysis['response'] == "Yes"]

# Group the filtered dataframe by 'policy_type' and 'gender' and calculate statistics
grouped_data = filtered_data.groupby(['policy_type', 'gender'])['total_claim_amount']

# Calculate mean, median, and variance
mean_total_claim = grouped_data.mean()
median_total_claim = grouped_data.median()
var_total_claim = grouped_data.var()

# Display the results
print("Mean Total Claim Amount by Policy Type and Gender:")
print(mean_total_claim)

print("\nMedian Total Claim Amount by Policy Type and Gender:")
print(median_total_claim)

print("\nVariance Total Claim Amount by Policy Type and Gender:")
print(var_total_claim)

Mean Total Claim Amount by Policy Type and Gender:
policy_type     gender
Corporate Auto  F         433.738499
                M         408.582459
Personal Auto   F         452.965929
                M         457.010178
Special Auto    F         453.280164
                M         429.527942
Name: total_claim_amount, dtype: float64

Median Total Claim Amount by Policy Type and Gender:
policy_type     gender
Corporate Auto  F         420.041981
                M         364.800000
Personal Auto   F         424.330166
                M         412.800000
Special Auto    F         420.041981
                M         345.600000
Name: total_claim_amount, dtype: float64

Variance Total Claim Amount by Policy Type and Gender:
policy_type     gender
Corporate Auto  F         48822.219045
                M         45595.721839
Personal Auto   F         67702.398260
                M         52920.431102
Special Auto    F         55175.319200
                M         62386.916734
Name: tota

In [18]:
#checking
unique_policies = data_mk_cust_analysis['policy'].unique()
print("Unique values in the 'policy' column:")
print(unique_policies)

null_policy_count = data_mk_cust_analysis['policy'].isnull().sum()
print("\nNumber of null values in the 'policy' column:")
print(null_policy_count)

Unique values in the 'policy' column:
['Corporate L3' 'Personal L3' 'Personal L2' 'Corporate L2' 'Personal L1'
 'Special L1' 'Corporate L1' 'Special L3' 'Special L2']

Number of null values in the 'policy' column:
0


In [20]:
#Grouping data by state and policy to get the number of customers 

grouped_counts = data_mk_cust_analysis.groupby(['state', 'policy']).size().reset_index(name='count')
# checking the results
#print("Counts of occurrences grouped by state and policy:")
#print(grouped_counts)


Counts of occurrences grouped by state and policy:
         state        policy  count
0      Arizona  Corporate L1     62
1      Arizona  Corporate L2    119
2      Arizona  Corporate L3    200
3      Arizona   Personal L1    293
4      Arizona   Personal L2    434
5      Arizona   Personal L3    742
6      Arizona    Special L1     25
7      Arizona    Special L2     35
8      Arizona    Special L3     27
9   California  Corporate L1    199
10  California  Corporate L2    276
11  California  Corporate L3    499
12  California   Personal L1    514
13  California   Personal L2   1043
14  California   Personal L3   1501
15  California    Special L1     22
16  California    Special L2     63
17  California    Special L3     66
18      Nevada  Corporate L1     37
19      Nevada  Corporate L2     73
20      Nevada  Corporate L3    109
21      Nevada   Personal L1    136
22      Nevada   Personal L2    212
23      Nevada   Personal L3    391
24      Nevada    Special L1      5
25      Nevad

In [22]:
#Grouping data by state and filter data to keep groups of 500 cutomers at least
grouped_counts2 = data_mk_cust_analysis.groupby(['state']).size().reset_index(name='count')

# Checking by displaying the results
print("Counts of occurrences grouped by state:")
print(grouped_counts2)


Counts of occurrences grouped by state:
        state  count
0     Arizona   1937
1  California   4183
2      Nevada    993
3      Oregon   2909
4  Washington    888


In [23]:
#There is no state with less tan 5000 customers in our dataset, but if it was the case we could do as follow:
# Grouping by 'state' and count the occurrences to identify the state to keep
state_counts = data_mk_cust_analysis.groupby('state').size()

# Filter states with counts > 500 and get their name
states_over_500 = state_counts[state_counts > 500].index

# Filter the original DataFrame to only keep rows with selected states
filtered_data = data_mk_cust_analysis[data_mk_cust_analysis['state'].isin(states_over_500)]

In [4]:
# Group the filtered dataframe by 'education' and 'gender' and calculate statistics
grouped_data4 = data_mk_cust_analysis.groupby(['education', 'gender'])['customer_lifetime_value']

# Calculate mean, median, and variance
median_total_clv4 = grouped_data4.median()
max_total_clv4 = grouped_data4.max()
min_total_clv4 = grouped_data4.min()
range_total_clv4=grouped_data4.max() - grouped_data4.min()

# Display the results
print("\nMedian customer lifetime value by Education and Gender:")
print(median_total_clv4)

print("\nMaximal customer lifetime value by Education and Gender:")
print(max_total_clv4)

print("\nMinimal customer lifetime value by Education and Gender:")
print(min_total_clv4)

print("\nRange customer lifetime value by Education and Gender:")
print(range_total_clv4)


Median customer lifetime value by Education and Gender:
education             gender
Bachelor              F         5640.505303
                      M         5548.031892
College               F         5623.611187
                      M         6005.847375
Doctor                F         5332.462694
                      M         5577.669457
High School or Below  F         6039.553187
                      M         6286.731006
Master                F         5729.855012
                      M         5579.099207
Name: customer_lifetime_value, dtype: float64

Maximal customer lifetime value by Education and Gender:
education             gender
Bachelor              F         73225.95652
                      M         67907.27050
College               F         61850.18803
                      M         61134.68307
Doctor                F         44856.11397
                      M         32677.34284
High School or Below  F         55277.44589
                      M         8

## Bonus

5. The marketing team wants to analyze the number of policies sold by state and month. Present the data in a table where the months are arranged as columns and the states are arranged as rows.

6.  Display a new DataFrame that contains the number of policies sold by month, by state, for the top 3 states with the highest number of policies sold.

*Hint:*
- *To accomplish this, you will first need to group the data by state and month, then count the number of policies sold for each group. Afterwards, you will need to sort the data by the count of policies sold in descending order.*
- *Next, you will select the top 3 states with the highest number of policies sold.*
- *Finally, you will create a new DataFrame that contains the number of policies sold by month for each of the top 3 states.*

7. The marketing team wants to analyze the effect of different marketing channels on the customer response rate.

Hint: You can use melt to unpivot the data and create a table that shows the customer response rate (those who responded "Yes") by marketing channel.

External Resources for Data Filtering: https://towardsdatascience.com/filtering-data-frames-in-pandas-b570b1f834b9

In [ ]:
# your code goes here